In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp submission

<IPython.core.display.Javascript object>

# Submission

In [ ]:
#hide
from nbdev.showdoc import *

<IPython.core.display.Javascript object>

In [ ]:
#export
import uuid
import numpy as np
import pandas as pd
from pathlib import Path
from typeguard import typechecked
from abc import abstractmethod
from rich import print as rich_print
from numerapi import NumerAPI, SignalsAPI

from numerai_blocks.download import BaseIO
from numerai_blocks.key import Key

<IPython.core.display.Javascript object>

## 1. Base

In [ ]:
#export
@typechecked
class BaseSubmittor(BaseIO):
    def __init__(self, directory_path: str, key: Key):
        super(BaseSubmittor, self).__init__(directory_path)
        self.dir = Path(directory_path)
        self._create_directory()
        self.key = key

    @abstractmethod
    def save_csv(self, dataf: pd.DataFrame, file_name: str, target_columns: list, *args, **kwargs):
        """ Save DataFrame with 'id' + 'target_columns'. """
        ...

    @abstractmethod
    def upload_predictions(self, file_name: str, model_name: str, *args, **kwargs):
        """ Upload file for given model name through API. """
        ...

    def full_submission(self, dataf: pd.DataFrame, file_name: str, model_name: str, target_columns: list, *args, **kwargs):
        """ Save DataFrame and upload predictions through API. """
        self.save_csv(dataf=dataf, file_name=file_name, target_columns=target_columns, *args, **kwargs)
        self.upload_predictions(file_name=file_name, model_name=model_name, *args, **kwargs)

    def __call__(self, dataf: pd.DataFrame, file_name: str, model_name: str, target_columns: list, *args, **kwargs):
        """
        The most common use case will be to create a CSV and submit it immediately after that.
        full_submission handles this.
        """
        self.full_submission(dataf=dataf,
                             file_name=file_name,
                             model_name=model_name,
                             target_columns=target_columns,
                             *args, **kwargs)

<IPython.core.display.Javascript object>

## 2. Numerai Classic

In [ ]:
#export
class NumeraiClassicSubmittor(BaseSubmittor):
    """
    Submit for Numerai Classic through NumerAPI
    :param directory_path: Base directory to save and read prediction files from.
    :param key: Key object (numerai-blocks.key.Key) containing valid credentials for Numerai Classic.
    *args, **kwargs will be passed to NumerAPI initialization.
    """
    def __init__(self, directory_path: str, key: Key, *args, **kwargs):
        super(NumeraiClassicSubmittor, self).__init__(directory_path=directory_path, key=key)
        self.napi = NumerAPI(public_id=self.key.pub_id, secret_key=self.key.secret_key, *args, **kwargs)

    def save_csv(self, dataf: pd.DataFrame, file_name: str, target_columns: list, *args, **kwargs):
        full_path = str(self.dir / file_name)
        rich_print(f":page_facing_up: Saving predictions CSV to '{full_path}'. :page_facing_up:")
        dataf[target_columns].to_csv(full_path, *args, **kwargs)

    def upload_predictions(self, file_name: str, model_name: str, *args, **kwargs):
        full_path = str(self.dir / file_name)
        model_id = self._get_model_id(model_name=model_name)
        rich_print(f":airplane: Uploading predictions from '{full_path}' for model [bold blue]'{model_name}'[/bold blue] (model_id='{model_id}') :airplane:")
        self.napi.upload_predictions(file_path=full_path,
                                     model_id=model_id,
                                     *args, **kwargs)
        rich_print(f":thumbs_up: Submission of '{full_path}' for [bold blue]{model_name}[/bold blue] is successful! :thumbs_up:")

    def _get_model_id(self, model_name: str) -> str:
        """ Get ID needed for prediction uploading. """
        return self.get_model_mapping()[model_name]

    @property
    def get_model_mapping(self) -> dict:
        """ Mapping between raw model names and model IDs. """
        return self.napi.get_models()

<IPython.core.display.Javascript object>

In [ ]:
# Initialization (Random credentials)
classic_key = Key(pub_id="UFVCTElDX0lE", secret_key="U1VQRVJfU0VDUkVUX0tFWQ==")
num_sub = NumeraiClassicSubmittor(directory_path="test_sub",
                                  key=classic_key)
assert num_sub.dir.is_dir()

# Create random dataframe
n_rows, n_columns = 100, 20
targets = [f"target_{i}" for i in range(n_columns)]
dataf = pd.DataFrame(np.random.uniform(size=(n_rows, n_columns)), columns=targets)
dataf['id'] = [uuid.uuid4() for _ in range(n_rows)]
dataf = dataf.set_index('id')
dataf.head(1)

🔑 Numerai Auth key initialized with pub_id = 'UFVCTElDX0lE' 🔑

,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19
id,,,,,,,,,,,,,,,,,,,,
dcd99619-af4c-46f5-bdba-5fd9bfbdbc2a,0.356854,0.210242,0.69456,0.612327,0.068831,0.424903,0.681421,0.108476,0.686033,0.449581,0.721593,0.91747,0.692445,0.935427,0.71167,0.006138,0.98922,0.663762,0.806196,0.258941


<IPython.core.display.Javascript object>

In [ ]:
# Save CSV and upload
# num_sub.full_submission(dataf=dataf, file_name='test.csv', target_columns=targets, model_name="test")

<IPython.core.display.Javascript object>

In [ ]:
num_sub.save_csv(dataf=dataf, file_name='test.csv', target_columns=targets)
pd.read_csv("test_sub/test.csv").head(2)

📄 Saving predictions CSV to 'test_sub/test.csv'. 📄

,id,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,...,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19
0,dcd99619-af4c-46f5-bdba-5fd9bfbdbc2a,0.356854,0.210242,0.694560,0.612327,0.068831,0.424903,0.681421,0.108476,0.686033,...,0.721593,0.91747,0.692445,0.935427,0.711670,0.006138,0.989220,0.663762,0.806196,0.258941
1,8864ba30-1685-4a58-a417-b6820cd5d275,0.057623,0.698108,0.880435,0.946488,0.195565,0.542124,0.814636,0.239006,0.773203,...,0.518857,0.17776,0.151123,0.736674,0.926641,0.436305,0.023097,0.938972,0.065533,0.607959


<IPython.core.display.Javascript object>

## 3. Numerai Signals

<IPython.core.display.Javascript object>

In [ ]:
#hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script; notebook2script()

Converted 00_download.ipynb.
Converted 01_dataloaders.ipynb.
Converted 02_dataset.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04a_model.ipynb.
Converted 04b_modelpipeline.ipynb.
Converted 05_postprocessing.ipynb.
Converted 06_prediction_dataset.ipynb.
Converted 07_evaluation.ipynb.
Converted 08_key.ipynb.
Converted 09_submission.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>